In [1]:
from collections import Counter
from io import StringIO

import MeCab
import pandas as pd

In [2]:
tagger = MeCab.Tagger("-O wakati")
result = tagger.parse("Axrossはつかえる知識をともに学べる場所です")
print(result)
print(type(result))

Axross は つかえる 知識 を とも に 学べる 場所 です 

<class 'str'>


In [3]:
sentence = """吾輩は猫である。名前はまだ無い。
どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。
吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。
この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。
ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。
掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。
第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢あったがこんな片輪には一度も出会わした事がない。
のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。
どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。"""

In [4]:
tagger = MeCab.Tagger()
buffer = StringIO(tagger.parse(sentence))

df = pd.read_csv(
    buffer,
    names=["表層形", "発音", "読み", "原形", "品詞", "品詞再分類1", "品詞細分類2", "品詞細分類3", "活用型", "活用形"],
    skipfooter=1,
    sep="[\t,,]",
    engine="python",
)

print(df.count())
df.head()

表層形       295
発音        278
読み        278
原形        295
品詞        295
品詞再分類1     92
品詞細分類2     92
品詞細分類3    146
活用型        19
活用形         0
dtype: int64


,表層形,発音,読み,原形,品詞,品詞再分類1,品詞細分類2,品詞細分類3,活用型,活用形
0,吾輩,ワガハイ,ワガハイ,我が輩,代名詞,NaN,NaN,0.0,NaN,NaN
1,は,ワ,ハ,は,助詞-係助詞,None,None,NaN,NaN,NaN
2,猫,ネコ,ネコ,猫,名詞-普通名詞-一般,NaN,NaN,1.0,NaN,NaN
3,で,デ,ダ,だ,助動詞,助動詞-ダ,連用形-一般,NaN,NaN,NaN
4,ある,アル,アル,有る,動詞-非自立可能,五段-ラ行,終止形-一般,1.0,NaN,NaN


In [5]:
# noun_df = df.query("品詞=='名詞'")
noun_df = df[df["品詞"].str.contains("名詞") & ~df["品詞"].str.contains("代名詞")]
noun_df.head()

,表層形,発音,読み,原形,品詞,品詞再分類1,品詞細分類2,品詞細分類3,活用型,活用形
2,猫,ネコ,ネコ,猫,名詞-普通名詞-一般,NaN,NaN,1.0,NaN,NaN
6,名前,ナマエ,ナマエ,名前,名詞-普通名詞-一般,NaN,NaN,0.0,NaN,NaN
18,見当,ケントー,ケントウ,見当,名詞-普通名詞-一般,NaN,NaN,3.0,NaN,NaN
30,所,トコロ,トコロ,所,名詞-普通名詞-副詞可能,NaN,NaN,3.0,0.0,NaN
37,事,コト,コト,事,名詞-普通名詞-一般,NaN,NaN,2.0,NaN,NaN


In [6]:
noun_counter = dict()

# for word in noun_df["表層形"]:
#   if noun_counter.get(word):
#     noun_counter[word] += 1
#   else:
#     noun_counter[word] = 1


for word in noun_df["表層形"]:
    noun_counter[word] = 1 + noun_counter.get(word, 0)
print(noun_counter)

{'猫': 2, '名前': 1, '見当': 1, '所': 1, '事': 3, '記憶': 1, '人間': 4, 'もの': 4, 'あと': 1, '書生': 3, '中': 2, '獰悪': 1, '種族': 1, 'そう': 1, '時々': 2, '話': 1, '当時': 1, '考': 1, '掌': 2, 'スー': 1, '時': 2, '感じ': 2, '上': 1, '顔': 3, '見': 1, '始': 1, '妙': 1, '今': 1, '一': 2, '毛': 1, '装飾': 1, 'はず': 1, '薬缶': 1, '後': 1, '逢': 1, '片輪': 1, '度': 1, '真中': 1, '突起': 1, '穴': 1, '煙': 1, '実': 1, '煙草': 1, 'この頃': 1}


In [7]:
# def sortkey(v):
#     return v[1]
# sorted(noun_counter.items(), key=sortkey, reverse=True)[:10]

print(sorted(noun_counter.items(), key=lambda x: x[1], reverse=True)[:10])
# print(sorted(noun_counter.items(), key=lambda x: x[1], reverse=True)[0:10])
# print(sorted(noun_counter.items(), key=lambda x: x[1], reverse=True)[1:10])

[('人間', 4), ('もの', 4), ('事', 3), ('書生', 3), ('顔', 3), ('猫', 2), ('中', 2), ('時々', 2), ('掌', 2), ('時', 2)]


In [8]:
from collections import Counter

c = Counter(noun_df["表層形"])
print(c.most_common(10))

[('人間', 4), ('もの', 4), ('事', 3), ('書生', 3), ('顔', 3), ('猫', 2), ('中', 2), ('時々', 2), ('掌', 2), ('時', 2)]


In [9]:
sentence2 = """親譲りの無鉄砲で小供の時から損ばかりしている。小学校に居る時分学校の二階から飛び降りて一週間ほど腰を抜かした事がある。なぜそんな無闇をしたと聞く人があるかも知れぬ。別段深い理由でもない。新築の二階から首を出していたら、同級生の一人が冗談に、いくら威張っても、そこから飛び降りる事は出来まい。弱虫やーい。と囃したからである。小使に負ぶさって帰って来た時、おやじが大きな眼をして二階ぐらいから飛び降りて腰を抜かす奴があるかと云ったから、この次は抜かさずに飛んで見せますと答えた。
　親類のものから西洋製のナイフを貰って奇麗な刃を日に翳して、友達に見せていたら、一人が光る事は光るが切れそうもないと云った。切れぬ事があるか、何でも切ってみせると受け合った。そんなら君の指を切ってみろと注文したから、何だ指ぐらいこの通りだと右の手の親指の甲をはすに切り込んだ。幸ナイフが小さいのと、親指の骨が堅かったので、今だに親指は手に付いている。しかし創痕は死ぬまで消えぬ。
　庭を東へ二十歩に行き尽すと、南上がりにいささかばかりの菜園があって、真中に栗の木が一本立っている。これは命より大事な栗だ。実の熟する時分は起き抜けに背戸を出て落ちた奴を拾ってきて、学校で食う。菜園の西側が山城屋という質屋の庭続きで、この質屋に勘太郎という十三四の倅が居た。勘太郎は無論弱虫である。弱虫の癖に四つ目垣を乗りこえて、栗を盗みにくる。ある日の夕方折戸の蔭に隠れて、とうとう勘太郎を捕まえてやった。その時勘太郎は逃げ路を失って、一生懸命に飛びかかってきた。向うは二つばかり年上である。弱虫だが力は強い。鉢の開いた頭を、こっちの胸へ宛ててぐいぐい押した拍子に、勘太郎の頭がすべって、おれの袷の袖の中にはいった。邪魔になって手が使えぬから、無暗に手を振ったら、袖の中にある勘太郎の頭が、右左へぐらぐら靡いた。しまいに苦しがって袖の中から、おれの二の腕へ食い付いた。痛かったから勘太郎を垣根へ押しつけておいて、足搦をかけて向うへ倒してやった。山城屋の地面は菜園より六尺がた低い。勘太郎は四つ目垣を半分崩して、自分の領分へ真逆様に落ちて、ぐうと云った。勘太郎が落ちるときに、おれの袷の片袖がもげて、急に手が自由になった。その晩母が山城屋に詫びに行ったついでに袷の片袖も取り返して来た。
　この外いたずらは大分やった。大工の兼公と肴屋の角をつれて、茂作の人参畠をあらした事がある。人参の芽が出揃わぬ処へ藁が一面に敷いてあったから、その上で三人が半日相撲をとりつづけに取ったら、人参がみんな踏みつぶされてしまった。古川の持っている田圃の井戸を埋めて尻を持ち込まれた事もある。太い孟宗の節を抜いて、深く埋めた中から水が湧き出て、そこいらの稲にみずがかかる仕掛であった。その時分はどんな仕掛か知らぬから、石や棒ちぎれをぎゅうぎゅう井戸の中へ挿し込んで、水が出なくなったのを見届けて、うちへ帰って飯を食っていたら、古川が真赤になって怒鳴り込んで来た。たしか罰金を出して済んだようである。
　おやじはちっともおれを可愛がってくれなかった。母は兄ばかり贔屓にしていた。この兄はやに色が白くって、芝居の真似をして女形になるのが好きだった。おれを見る度にこいつはどうせ碌なものにはならないと、おやじが云った。乱暴で乱暴で行く先が案じられると母が云った。なるほど碌なものにはならない。ご覧の通りの始末である。行く先が案じられたのも無理はない。ただ懲役に行かないで生きているばかりである。
　母が病気で死ぬ二三日前台所で宙返りをしてへっついの角で肋骨を撲って大いに痛かった。母が大層怒って、お前のようなものの顔は見たくないと云うから、親類へ泊りに行っていた。するととうとう死んだと云う報知が来た。そう早く死ぬとは思わなかった。そんな大病なら、もう少し大人しくすればよかったと思って帰って来た。そうしたら例の兄がおれを親不孝だ、おれのために、おっかさんが早く死んだんだと云った。口惜しかったから、兄の横っ面を張って大変叱られた。
　母が死んでからは、おやじと兄と三人で暮していた。おやじは何にもせぬ男で、人の顔さえ見れば貴様は駄目だ駄目だと口癖のように云っていた。何が駄目なんだか今に分らない。妙なおやじがあったもんだ。兄は実業家になるとか云ってしきりに英語を勉強していた。元来女のような性分で、ずるいから、仲がよくなかった。十日に一遍ぐらいの割で喧嘩をしていた。ある時将棋をさしたら卑怯な待駒をして、人が困ると嬉しそうに冷やかした。あんまり腹が立ったから、手に在った飛車を眉間へ擲きつけてやった。眉間が割れて少々血が出た。兄がおやじに言付けた。おやじがおれを勘当すると言い出した。"""

In [10]:
tagger = MeCab.Tagger()
result = StringIO(tagger.parse(sentence2))
df = pd.read_csv(
    result,
    names=["表層形", "発音", "読み", "原形", "品詞", "品詞再分類1", "品詞細分類2", "品詞細分類3", "活用型", "活用形"],
    skipfooter=1,
    sep="[\t,,]",
    engine="python",
)

df.head()

,表層形,発音,読み,原形,品詞,品詞再分類1,品詞細分類2,品詞細分類3,活用型,活用形
0,親譲り,オヤユズリ,オヤユズリ,親譲り,名詞-普通名詞-一般,NaN,NaN,3.0,NaN,NaN
1,の,ノ,ノ,の,助詞-格助詞,None,None,NaN,NaN,NaN
2,無鉄砲,ムテッポー,ムテッポウ,無鉄砲,名詞-普通名詞-形状詞可能,NaN,NaN,2.0,NaN,NaN
3,で,デ,ダ,だ,助動詞,助動詞-ダ,連用形-一般,NaN,NaN,NaN
4,小供,コドモ,コドモ,子供,名詞-普通名詞-一般,NaN,NaN,0.0,NaN,NaN


In [11]:
tagger = MeCab.Tagger()
buffer = StringIO(tagger.parse(sentence2))
df = pd.read_csv(
    buffer,
    names=["表層形", "発音", "読み", "原形", "品詞", "品詞再分類1", "品詞細分類2", "品詞細分類3", "活用型", "活用形"],
    skipfooter=1,
    sep="[\t,,]",
    engine="python",
)
noun_df = df[
    df["品詞"].str.contains("名詞", na=False) & ~df["品詞"].str.contains("代名詞", na=False)
]
# display(noun_df.head())
c = Counter(noun_df["表層形"])
print(c.most_common())

[('勘太郎', 9), ('おやじ', 8), ('兄', 7), ('事', 6), ('手', 6), ('母', 6), ('人', 5), ('中', 5), ('時', 4), ('二', 4), ('弱虫', 4), ('もの', 4), ('日', 4), ('学校', 3), ('時分', 3), ('階', 3), ('一', 3), ('親指', 3), ('菜園', 3), ('栗', 3), ('山城', 3), ('屋', 3), ('三', 3), ('四', 3), ('つ', 3), ('頭', 3), ('袷', 3), ('袖', 3), ('人参', 3), ('駄目', 3), ('腰', 2), ('一人', 2), ('奴', 2), ('親類', 2), ('ナイフ', 2), ('指', 2), ('通り', 2), ('今', 2), ('庭', 2), ('質屋', 2), ('十', 2), ('目', 2), ('垣', 2), ('向う', 2), ('片袖', 2), ('角', 2), ('古川', 2), ('井戸', 2), ('水', 2), ('仕掛', 2), ('乱暴', 2), ('顔', 2), ('眉間', 2), ('親譲り', 1), ('無鉄砲', 1), ('小供', 1), ('損', 1), ('週間', 1), ('理由', 1), ('新築', 1), ('首', 1), ('同級', 1), ('生', 1), ('冗談', 1), ('小使', 1), ('眼', 1), ('次', 1), ('西洋', 1), ('製', 1), ('刃', 1), ('友達', 1), ('注文', 1), ('右', 1), ('甲', 1), ('はす', 1), ('幸', 1), ('骨', 1), ('創痕', 1), ('東', 1), ('二十', 1), ('歩', 1), ('南', 1), ('上がり', 1), ('真中', 1), ('木', 1), ('本', 1), ('命', 1), ('実', 1), ('起き抜け', 1), ('背戸', 1), ('西側', 1), ('続き', 1), ('倅', 1), ('癖', 1), ('夕方', 

In [12]:
tagger = MeCab.Tagger(r'-d "/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"')
result = StringIO(tagger.parse("相馬ゼミにはインスタ映え女子がいない"))
df = pd.read_csv(
    result,
    names=["表層形", "品詞", "品詞細分類1", "品詞細分類2", "品詞細分類3", "活用型", "活用形", "原形", "読み", "発音"],
    skipfooter=1,
    sep="[\t,,]",
    engine="python",
)

df

,表層形,品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音
0,相馬,名詞,固有名詞,人名,姓,*,*,相馬,ソウマ,ソーマ
1,ゼミ,名詞,一般,*,*,*,*,ゼミ,ゼミ,ゼミ
2,に,助詞,格助詞,一般,*,*,*,に,ニ,ニ
3,は,助詞,係助詞,*,*,*,*,は,ハ,ワ
4,インスタ映え,名詞,固有名詞,一般,*,*,*,インスタ映え,インスタハエ,インスタハエ
5,女子,名詞,一般,*,*,*,*,女子,ジョシ,ジョシ
6,が,助詞,格助詞,一般,*,*,*,が,ガ,ガ
7,いない,名詞,固有名詞,組織,*,*,*,いない,イナイ,イナイ


In [13]:
display(HTML("<h1>Test</h1>"))
display(HTML("<h2>test<h2>"))

NameError: name 'HTML' is not defined